In [1]:
import json
from datetime import datetime
import time
import numpy as np
import pandas as pd
import math

# sys.path.append(os.path.abspath("/content/SatoshiVault/code")) # Colab

from utilis import *

price = get_current_price("btcusdt")

# Regular
* Price
* Price variance
* Technical indicators (Choose those that make more sense and have less data)
* Funding Rate
* Longs Short Rations, total
* Total OI
* Total Put/call
* Volume
* .....

# Do heatmaps of:

## Spot
 * Books
 * Trades
 * Canceled Books
 * Reinforced books

## Perp/Futures
 * Books  --- minute, hours
 * Trades --- Minutes hours
 * Canceled Books 
 * Reinforced books
 * Open Interest
 * Funding Rate
 * Top Traders Accounts  -- hourly (Binance)
 * Top Traders Positions -- hourly (Binance)
 * Global Account Ratio --- hourly (Binance, Bybit, okx)

## Options
* Call OI by Strike ---- to Price
* Put Open Interest by strike --- To price
* And those generated from them

# 1 min Features

# 1 h features



# BOOKS AND VOLUME MUST BE AGGREGATED


In [ ]:
# Colab

# initial_books = json.load(open('/content/SatoshiVault/code/data_binance_books_Trades/books.json', 'r'))
# trades = json.load(open('/content/SatoshiVault/code/data_binance_books_Trades/trades.json', 'r'))
# books = json.load(open('/content/SatoshiVault/code/data_binance_books_Trades/bupdates.json', 'r'))

# Else
# initial_books = json.load(open('code/data_binance_books_Trades/data_binance_books_Trades/books.json', 'r'))
# trades = json.load(open('code/data_binance_books_Trades/trades.json', 'r'))
# books = json.load(open('code/data_binance_books_Trades/bupdates.json', 'r'))


a  = secondProcessor('binance', 'perpetual', 'btc_usdt', btc_price, 20)

start =time.time()
a.update_books(initial_books, 'bids', 'asks', 'timestamp')
a.dfs_input_books()
a.dfs_books.drop(columns=a.dfs_books.columns[a.dfs_books.eq(0).all()], inplace=True)

for e in books:
    a.update_books(e, 'b', 'a', 'timestamp')
    a.dfs_input_books()


print(f"elapsed_time for {len(books)+1} iterations: ", time.time() - start)



In [2]:
class secondProcessor:
    """
        insType : spot, perpetual, options
            Spot - processes books, trades, canceled books, and reinforced books
            Perpetual - processes books, trades, canceled books, and reinforced books, OI, liquidations and other Statistical data
            Option - processes only OI by strike, volume, liquidations
        start_price : the last price of the ticker . If you want to aggragate books from different exchanges, you should keep the current price consistent amount all of them
        bucket_range : price range of buckets to aggregate books
        n_buckets : number of buckets to create into single direction
        price_level_ceiling : % ceiling of price levels to ommit 5%

        BIG NOTE:  If the book is above 5% from the current price, it will be deleted for computational efficiency. 
                   It would be wise to assume the spoof 
    """

    def __init__(self, exchange, insType, symbol, start_price, level_range, price_level_ceiling=5):
        # Identification
        self.exchange = exchange
        self.insType = insType
        self.symbol = symbol
        self.level_range = level_range
        self.price_level_ceiling = price_level_ceiling
        self.level_ranges = np.arange(np.floor((start_price -  (level_range * np.ceil((start_price * price_level_ceiling * 0.01) / 20))) / level_range) * level_range, np.ceil((start_price +  (level_range * np.ceil((start_price * price_level_ceiling * 0.01) / 20))) / level_range) * level_range+1, level_range)
        self.B = {"timestamp": 1, "current_price": float(start_price), "bids" : {}, "asks" : {}}
        self.current_price = start_price
        self.current_second = int(time.time() % 60)  
        # Bucket ranges for dataframes
        self.current_buckets_dfs = get_bucket_list(start_price, level_range, level_range)
        # self.current_buckets_dfm = get_bucket_list(start_price, bucket_range, n_buckets)
        if insType == 'spot':
            # Raw data frames 1 sec
            self.dfs_books = create_data_frame('sec', self.level_ranges)
            self.dfs_trades = create_data_frame('sec', self.level_ranges)
            self.dfs_canceled_books = create_data_frame('sec', self.level_ranges)
            self.dfs_reinforced_books = create_data_frame('sec', self.level_ranges)
            # Raw data frames 1 sec
            # self.dfm_books = create_data_frame('min', self.current_buckets_dfm)
            # self.dfm_trades = create_data_frame('min', self.current_buckets_dfm)
            # self.dfm_canceled_books = create_data_frame('min', self.current_buckets_dfm)
            # self.dfm_reinforced_books = create_data_frame('min', self.current_buckets_dfm)
        if insType == 'perpetual':
            # Raw data frames 1 sec
            self.dfs_books = create_data_frame('sec', self.level_ranges)
            self.dfs_trades = create_data_frame('sec', self.level_ranges)
            self.dfs_canceled_books = create_data_frame('sec', self.level_ranges)
            self.dfs_reinforced_books = create_data_frame('sec', self.level_ranges)
            self.dfs_OI = create_data_frame('sec', self.level_ranges)
            # Raw data frames 1 sec
            # self.dfm_books = create_data_frame('min', self.current_buckets_dfm)
            # self.dfm_trades = create_data_frame('min', self.current_buckets_dfm)
            # self.dfm_canceled_books = create_data_frame('min', self.current_buckets_dfm)
            # self.dfm_reinforced_books = create_data_frame('min', self.current_buckets_dfm)
            # self.dfm_binance_GLSA = create_data_frame('min', self.current_buckets_dfm)
            # self.dfm_OI = create_data_frame('min', self.current_buckets_dfm)
            # self.dfm_okx_GLSA = create_data_frame('min', self.current_buckets_dfm)
            # self.dfm_bybit_GLSA = create_data_frame('min', self.current_buckets_dfm)
            # self.dfm_binance_TLSA = create_data_frame('min', self.current_buckets_dfm)
            # self.dfm_binance_TLSP = create_data_frame('min', self.current_buckets_dfm)
        if insType == 'options':
            # Add here second frame
            self.dfm_deribit_call_OI_strike = create_data_frame('min', self.level_ranges)
            self.dfm_deribit_Put_OI_strike = create_data_frame('min', self.level_ranges)
        # Features
        #self.features_min = create_data_frame('min', self.current_buckets_dfh)  # Redo, different columns
        #self.features_hour = create_data_frame('h', self.current_buckets_dfh)   # Redo, different columns other module
        # self.current_buckets_dfh = get_bucket_list(start_price, bucket_range, n_buckets)

    def update_current_price(self, price):
        self.current_price = price

    
    def update_books(self, total_books, bids_name, asks_name, t_name):
        """
            bids_name, asks_name, t_name : Different jsons have different name for bids and asks and timestamps
            t__name as if time.time()
        """
        self.update_books_helper(total_books[bids_name], 'bids')
        self.update_books_helper(total_books[asks_name], 'asks')
        self.B['timestamp'] = total_books[t_name]
        self.B['current_price'] = (max([float(x[0]) for x in total_books[bids_name]]) + min([float(x[0]) for x in total_books[asks_name]])) / 2

    def update_books_helper(self, books, side):
        """
          side: bids, asks
        """
        # Omit books above 5% from the current price
        for book in books:
            p = float(book[0])
            cp = float(self.B['current_price'])
            if percentage_difference(p, cp) > self.price_level_ceiling:
                continue
            if book[1] == "0" or book[1] == 0:
                del self.B[side][book[0]]
            else:
                self.B[side][book[0]] = book[1]

    def dfs_input_books(self):
        """
            Inputs bids and asks into dfs
        """
        # Raw data
        current_second = int(self.B['timestamp'] % 60)  
        print(current_second)
        current_price = (np.max([float(x) for x in self.B['bids'].keys()]) + np.min([float(x) for x in self.B['asks'].keys()])) / 2
        raw_books_quatities = np.array([float(x) for x in self.B['bids'].values()] + [float(x) for x in self.B['asks'].values()])
        raw_books_levels = np.array([float(x) for x in self.B['bids'].keys()] + [float(x) for x in self.B['asks'].keys()])
        # 
        self.dfs_books['price'] = current_price
        # New books levels
        start = np.floor(np.min(raw_books_levels) / self.level_range ) * self.level_range 
        end = np.ceil(np.max(raw_books_levels) / self.level_range ) * self.level_range 
        books_levels = np.arange(start, end+1, self.level_range)
        # Are there new levels currently not in dataframe?
        new_levels = np.setdiff1d(books_levels, self.level_ranges)
        # Indices and grouped values
        grouped_values = np.bincount(np.digitize(raw_books_levels, bins=books_levels), weights=raw_books_quatities)
        if new_levels.size == 0:
            self.dfs_books.loc[current_second, books_levels] = grouped_values
            self.dfs_books.fillna(method='ffill',  inplace=True)
        else:
            # Create newcolumns pandas dataframe
            self.dfs_books[new_levels] = float(0)
            self.dfs_trades[new_levels] = float(0)
            self.dfs_canceled_books[new_levels] = float(0)
            self.dfs_reinforced_books[new_levels] = float(0)
            # Input new values
            self.dfs_books.loc[current_second, books_levels] = grouped_values
            self.dfs_books.fillna(method='ffill',  inplace=True)
            # Remove columns if those are not within 5% range
            columns_to_drop = filter_ranges(self.level_ranges, current_price, self.price_level_ceiling)
            self.dfs_books.drop(columns=columns_to_drop, inplace=True)
            self.dfs_trades.drop(columns=columns_to_drop, inplace=True)
            self.dfs_canceled_books.drop(columns=columns_to_drop, inplace=True)
            self.dfs_reinforced_books.drop(columns=columns_to_drop, inplace=True)
            self.level_ranges = np.setdiff1d(np.concatenate((self.level_ranges, new_levels)), columns_to_drop)



    def dfs_input_trades(self, trade, t_name, p_name, q_name):                                            # Works fine  # FIX when inputing new data to dfm The last raw is being deleted
        """ 
            t_name: timestamp name in the dictionary
            p_name: price name in the dictionary
            q_name: quantity name in the dictionary

            Inputs price, volume(amount) and trades into dfs_trades frame
            Inputs price into dfs_books
        """
        current_second = int(trade[t_name]% 60)  
        current_price = float(trade[p_name])
        amount = float(trade['q'])
        level = find_level(self.level_ranges, current_price)
        # Not needed, just incase
        if level == None:
            new_levels = np.setdiff1d(level, self.level_ranges)
            # Create newcolumns pandas dataframe
            self.dfs_books[new_levels] = float(0)
            self.dfs_trades[new_levels] = float(0)
            self.dfs_canceled_books[new_levels] = float(0)
            self.dfs_reinforced_books[new_levels] = float(0)
            # Input new values
            self.dfs_trades.loc[current_second, level] += amount
            # The next levels should all be 0
            if current_second == 59:
                # Push to 1 min frame
                self.dfs_trades.loc[0:58] = float(0)
            if current_second == 0:
                self.dfs_trades.loc[59] = float(0)

        else:
            self.dfs_trades.loc[current_second, level] += amount
            if current_second == 59:
                self.dfs_trades.loc[0] = float(0)
            else:
                self.dfs_trades.loc[current_second] = float(0)





    # def dfs_input_canceled_reinforced(self):
    #     """
    #         Periodic task, must be done every minute
    #     """
    #     # Calculate canceled books
    #     current_second = get_current_time('s')
    #     previous_column = current_second -1
    #     if current_second == 0:
    #         previous_column = 59
    #     # Calculates canceled or reinforced books over certain timestamps
    #     D = (self.dfs_books.iloc[current_second, :] + self.dfs_trades.iloc[current_second, :]) - (self.dfs_books.iloc[previous_column, :] + self.dfs_trades.iloc[previous_column, :])
    #     self.dfs_canc_books.iloc[current_second, :] = D.clip(lower=0)
    #     self.dfs_rein_books.iloc[current_second, :] = D.clip(upper=0).abs()


    # def dfs_input_trades(self, trade):                                            # Works fine  # FIX when inputing new data to dfm The last raw is being deleted
    #     """ 
    #         Inputs price, volume(amount) and trades into dfs_trades frame
    #         Inputs price into dfs_books
    #     """
    #     current_second = get_current_time('s')
    #     current_price = float(trade['p'])
    #     print(current_price)
    #     amount = float(trade['q'])
    #     located_bucket = self.locate_bucket(current_price, self.current_buckets_dfs)

    #     self.dfs_books.loc[current_second, 'price'] = current_price
    #     self.dfs_trades.loc[current_second, 'price'] = current_price
    #     self.dfs_canceled_books.loc[current_second, 'price'] = current_price
    #     self.dfs_reinforced_books.loc[current_second, 'price'] = current_price

    #     # If the bucket exists
    #     if located_bucket != None:
    #         col_name = "_".join([str(located_bucket[0]), str(located_bucket[1])])
    #         if np.isnan(self.dfs_trades.at[current_second, col_name]) == True:
    #             self.dfs_trades.loc[current_second, col_name] =  amount
    #         else: 
    #             self.dfs_trades.loc[current_second, col_name] = self.dfs_trades.at[current_second, col_name] + amount
    #     # if the bucket doesnt exist
    #     if located_bucket == None:
    #       # Makes new buckets
    #       new_buckets = self.get_new_buckets(current_price, self.current_buckets_dfs)
    #       self.current_buckets_dfs = sorted(self.current_buckets_dfs + new_buckets)
    #       new_names = ["_".join([str(x[0]), str(x[1])]) for x in new_buckets]
    #       for name in new_names:
    #           self.dfs_trades[name] = np.nan
    #           self.dfs_books[name] = np.nan
    #           self.dfs_canc[name] = np.nan
    #           self.dfs_rein[name] = np.nan
    #       # Finaly input values
    #       l = self.locate_bucket(current_price, self.current_buckets_dfs)
    #       col_name = "_".join([str(l[0]), str(l[1])])
    #       if np.isnan(self.dfs_trades.at[current_second, col_name]) == True:
    #           self.dfs_trades.loc[current_second, col_name] =  amount
    #       else: 
    #           self.dfs_trades.loc[current_second, col_name] = self.dfs_trades.at[current_second, col_name] + amount

    #     # Make sure the next row is empy
    #     s = get_current_time('s')
    #     if s == 59:
    #       s = 0
    #     dfs_trades.iloc[s+1, :] = np.nan




    # def dfs_input_books(self, books : dict):
    #     """
    #         Updates dfs bids and asks.
    #         The first time you call it, it populates entire dfs wthich will later be replaced
    #         Note: For consistency, start streaming at the first second
    #     """
    #     current_second = int(books['timestamp'] % 60)  # current_second = get_current_secod() # For production
    #     next_secod = 0 if current_second == 59 else current_second + 1
    #     if self.dfs_input_books_call_caount == 0:
    #         self.dfs_input_books_helper(books['asks'], current_second)
    #         self.dfs_input_books_helper(books['bids'], current_second)
    #         self.dfs_books = self.dfs_books.applymap(lambda x: np.nan if x == 0 else x).ffill().bfill()
    #         self.dfs_input_books_call_caount += 1
    #     else:
    #         self.dfs_input_books_helper(books['asks'], current_second)
    #         self.dfs_input_books_helper(books['bids'], current_second)
    #     #self.dfs_books.iloc[next_secod, :] = 0

    # def dfs_input_books_helper(self, books : list, current_second):                         # Works Fine # Works fine  # FIX when inputing new data to dfm. The last raw delete, which is not suppoused to be
    #     """
    #         Iputs books in dfs dataframe
    #         Creates new_buckets if necessary
    #         Updates current buckets dfs
    #     """
    #     for book in books:
    #         col = float(book[0])
    #         amount = float(book[1])
    #         located_bucket = self.locate_bucket(col, self.current_buckets_dfs)
    #         # If the bucket exists
    #         if located_bucket != None:
    #             col_name = "_".join([str(located_bucket[0]), str(located_bucket[1])])
    #             self.dfs_books.loc[current_second, col_name] +=  amount    # This are buckets, this must no be replace, but, rathre , add
    #         # If the bucket doesn't exist
    #         if located_bucket == None:
    #             # Makes new buckets
    #             new_buckets = self.get_new_buckets(col, self.current_buckets_dfs)
    #             self.current_buckets_dfs = sorted(self.current_buckets_dfs + new_buckets)
    #             new_names = ["_".join([str(x[0]), str(x[1])]) for x in new_buckets]
    #             for name in new_names:
    #                 self.dfs_trades[name] = float(0)
    #                 self.dfs_books[name] = float(0)
    #                 self.dfs_canceled_books[name] = float(0)
    #                 self.dfs_reinforced_books[name] = float(0)
    #             # Finaly input values
    #             l = self.locate_bucket(col, self.current_buckets_dfs)
    #             col_name = "_".join([str(l[0]), str(l[1])])
    #             self.dfs_books.loc[current_second, col_name] +=  amount


    # def locate_bucket(self, value: float, ranges : list):               # Works well
    #     for index, (start, end) in enumerate(ranges):
    #         if start <= value < end:
    #             return ranges[index]
    #     return None
      
            
    # def get_new_buckets(self, value : float, current_buckets : list):    # Works fine
    #     step = self.bucket_range
    #     mmax = max(current_buckets)[1]
    #     mmin = min(current_buckets)[0]
    #     l = []
    #     if value > mmax or value == mmax:
    #         rounded_value = int(math.ceil(value / step) * step)
    #         for v in range(mmax, rounded_value, step):
    #             l.append([v, v+step ])
    #         if rounded_value == value and value == mmax:
    #             l.append([int(value), int(value+step)])
    #     if value < mmin:
    #         rounded_value = max(0, int(value / step) * step)
    #         for v in range(rounded_value, mmin, step):
    #             l.append([v, v+step ])
    #     return l    

    # def dfs_remove_empty_columns(self):
    #     pass
        # # REDO
        # # If all of the certain buckets cointain the same NULLS - remove
        # # dfs
        # empty_columns = [[int(x.split('_')[0]), int(x.split('_')[1])]for x in self.dfs_books.columns[self.dfs_books.isnull().all()].tolist()]
        # self.current_buckets_dfs = [x for x in self.current_buckets_dfs if x not in empty_columns]
        # self.dfs_books.dropna(axis=1, how='all')
        # self.dfs_canc_books(axis=1, how='all')



In [6]:
initial_books = json.load(open('data_binance_books_Trades/books.json', 'r'))
trades = json.load(open('data_binance_books_Trades/trades.json', 'r'))
books = json.load(open('data_binance_books_Trades/bupdates.json', 'r'))




engine = secondProcessor('binance', 'perpetual', 'btc_usdt', 43000, 10, 1)


start_time = time.time()

engine.dfs_input_books(initial_books)

print("Elapsed_time", time.time() - start_time)

Elapsed_time 0.5400278568267822


In [11]:
engine.dfs_books.replace(0, np.nan).bfill(limit=60)


,price,volume,42990_43000,43000_43010,42620_42630,42630_42640,42640_42650,42650_42660,42660_42670,42670_42680,...,42240_42250,42230_42240,42220_42230,42210_42220,42200_42210,42190_42200,42180_42190,42170_42180,42160_42170,42150_42160
0,NaN,NaN,0.25955,0.00015,0.00021,0.00025,0.00074,0.34994,0.00025,0.00013,...,0.05056,0.04535,0.61559,0.09615,23.43409,0.10735,5.58725,0.03428,0.12386,0.00105
1,NaN,NaN,0.25955,0.00015,0.00021,0.00025,0.00074,0.34994,0.00025,0.00013,...,0.05056,0.04535,0.61559,0.09615,23.43409,0.10735,5.58725,0.03428,0.12386,0.00105
2,NaN,NaN,0.25955,0.00015,0.00021,0.00025,0.00074,0.34994,0.00025,0.00013,...,0.05056,0.04535,0.61559,0.09615,23.43409,0.10735,5.58725,0.03428,0.12386,0.00105
3,NaN,NaN,0.25955,0.00015,0.00021,0.00025,0.00074,0.34994,0.00025,0.00013,...,0.05056,0.04535,0.61559,0.09615,23.43409,0.10735,5.58725,0.03428,0.12386,0.00105
4,NaN,NaN,0.25955,0.00015,0.00021,0.00025,0.00074,0.34994,0.00025,0.00013,...,0.05056,0.04535,0.61559,0.09615,23.43409,0.10735,5.58725,0.03428,0.12386,0.00105
5,NaN,NaN,0.25955,0.00015,0.00021,0.00025,0.00074,0.34994,0.00025,0.00013,...,0.05056,0.04535,0.61559,0.09615,23.43409,0.10735,5.58725,0.03428,0.12386,0.00105
6,NaN,NaN,0.25955,0.00015,0.00021,0.00025,0.00074,0.34994,0.00025,0.00013,...,0.05056,0.04535,0.61559,0.09615,23.43409,0.10735,5.58725,0.03428,0.12386,0.00105
7,NaN,NaN,0.25955,0.00015,0.00021,0.00025,0.00074,0.34994,0.00025,0.00013,...,0.05056,0.04535,0.61559,0.09615,23.43409,0.10735,5.58725,0.03428,0.12386,0.00105
8,NaN,NaN,0.25955,0.00015,0.00021,0.00025,0.00074,0.34994,0.00025,0.00013,...,0.05056,0.04535,0.61559,0.09615,23.43409,0.10735,5.58725,0.03428,0.12386,0.00105
9,NaN,NaN,0.25955,0.00015,0.00021,0.00025,0.00074,0.34994,0.00025,0.00013,...,0.05056,0.04535,0.61559,0.09615,23.43409,0.10735,5.58725,0.03428,0.12386,0.00105


In [34]:
int(initial_books['timestamp'] % 60)

12

In [70]:
float('0.33076')

0.33076